In [3]:
associations = {
    "Silent": {'Toilet': ['Theta_AF7'], 'Water': [], 'Yes': ['Theta_TP10']},
    "Pain": {'Silent': ['Theta_TP9', 'Theta_AF8', 'Alpha_AF8'], 'Toilet': ['Delta_AF8', 'Gamma_AF7', 'Gamma_TP10'], 'Water': ['Alpha_TP9', 'Beta_AF7', 'Gamma_AF7'], 'Yes': ['Delta_TP10', 'Alpha_TP10', 'Beta_AF8', 'Gamma_AF7', 'Gamma_AF8'], },
    "Doctor": {'Move': ['Beta_AF8'], 'No': ['Alpha_AF7', 'Gamma_TP10'], 'Silent': [], 'Pain': ['Beta_TP9', 'Beta_TP10', 'Gamma_TP9'], 'Toilet': ['Beta_AF8'], 'Water': ['Theta_TP9', 'Theta_AF7', 'Beta_AF8', 'Gamma_AF8', 'Gamma_TP10'], 'Yes': ['Theta_TP9', 'Alpha_TP9', 'Gamma_TP10']},
    "Move": {'No': ['Alpha_AF8', 'Alpha_TP10', 'Beta_AF7', 'Beta_TP10', 'Gamma_AF7', 'Gamma_AF8'], 'Pain': ['Delta_TP10', 'Alpha_TP10', 'Gamma_AF7'], 'Silent': ['Theta_TP10', 'Alpha_AF7'], 'Toilet': ['Beta_TP9', 'Beta_AF8', 'Gamma_TP9', 'Gamma_AF8'], 'Water': ['Beta_AF8', 'Beta_TP10', 'Gamma_AF8'], 'Yes': ['Delta_TP10', 'Theta_AF8', 'Theta_TP10', 'Alpha_AF8', 'Alpha_TP10', 'Beta_AF7', 'Beta_TP10', 'Gamma_AF7']},
    "No": {'Pain': ['Alpha_TP10', 'Gamma_AF7', 'Gamma_TP10'], 'Silent': [], 'Toilet': ['Theta_AF7', 'Alpha_AF8', 'Beta_TP10', 'Gamma_AF8', 'Gamma_TP10'], 'Water': ['Theta_TP10', 'Alpha_TP10', 'Beta_TP9', 'Beta_TP10', 'Gamma_AF8', 'Gamma_TP10'], 'Yes': ['Alpha_AF8', 'Alpha_TP10', 'Beta_TP9', 'Beta_AF7', 'Beta_TP10', 'Gamma_AF7', 'Gamma_TP10']},
    "Toilet": {'Water': ['Beta_TP10', 'Gamma_AF7'], 'Yes': ['Delta_AF7', 'Alpha_AF8']},
    "Water": {'Yes': ['Theta_TP9', 'Beta_TP10', 'Gamma_TP10']}
}

output_list = ['Doctor', 'Move', 'No', 'Pain', 'Silent', 'Toilet', 'Water', 'Yes']

In [7]:
from collections import Counter
from joblib import load
import pandas as pd

old_winner = 0

while old_winner < len(output_list):
    winner = -1
    start_winner = old_winner

    for j in range(old_winner + 1, len(output_list)):
        
        print("Opponent : ", j)
        if j > old_winner:
            model_filename = f'./Step-7 Model Generation/Speech/{output_list[old_winner]}_{output_list[j]}_randomforest_model.joblib'
            # Load the saved model
            loaded_model = load(model_filename)

            # preprocess test dataset
            new_data = pd.read_csv("final_app.csv")

            # remove features
            for features in associations[output_list[old_winner]][output_list[j]]:
                del new_data[features]

            print(new_data.columns)

            new_data.dropna(inplace=True)
            new_data = new_data[~(new_data == 0.0).any(axis=1)]
            new_data.drop_duplicates(inplace=True)

            for col in new_data.select_dtypes(include=['float64']).columns:
                new_data[col] = new_data[col].map(lambda x: f'{x:.6f}')

            new_data = new_data.iloc[5:-5]  # remove first and last 5 rows

            # Make predictions using the loaded model
            predictions = loaded_model.predict(new_data)

            # Count occurrences of each class in predictions
            class_counts = Counter(predictions)
            # Sort the counts in descending order
            sorted_counts = class_counts.most_common()
            print(sorted_counts)

            old_winner += 1
            # update winner to next layer
            winner = output_list.index(sorted_counts[0][0])
            print("New winner : ", winner)
            print()
            
            if old_winner != winner:
                old_winner = winner
        
    if old_winner == winner and winner == len(output_list) - 2:
        break
    if start_winner == winner:
        break

print("*** Predicted Class : ", output_list[winner])  # Expected : Pain  

Opponent :  1
Index(['Delta_TP9', 'Delta_AF7', 'Delta_AF8', 'Delta_TP10', 'Theta_TP9',
       'Theta_AF7', 'Theta_AF8', 'Theta_TP10', 'Alpha_TP9', 'Alpha_AF7',
       'Alpha_AF8', 'Alpha_TP10', 'Beta_TP9', 'Beta_AF7', 'Beta_TP10',
       'Gamma_TP9', 'Gamma_AF7', 'Gamma_AF8', 'Gamma_TP10'],
      dtype='object')
[('Doctor', 58), ('Move', 1)]
New winner :  0

Opponent :  2
Index(['Delta_TP9', 'Delta_AF7', 'Delta_AF8', 'Delta_TP10', 'Theta_TP9',
       'Theta_AF7', 'Theta_AF8', 'Theta_TP10', 'Alpha_TP9', 'Alpha_AF8',
       'Alpha_TP10', 'Beta_TP9', 'Beta_AF7', 'Beta_AF8', 'Beta_TP10',
       'Gamma_TP9', 'Gamma_AF7', 'Gamma_AF8'],
      dtype='object')
[('Doctor', 50), ('No', 9)]
New winner :  0

Opponent :  3
Index(['Delta_TP9', 'Delta_AF7', 'Delta_AF8', 'Delta_TP10', 'Theta_TP9',
       'Theta_AF7', 'Theta_AF8', 'Theta_TP10', 'Alpha_TP9', 'Alpha_AF7',
       'Alpha_AF8', 'Alpha_TP10', 'Beta_AF7', 'Beta_AF8', 'Gamma_AF7',
       'Gamma_AF8', 'Gamma_TP10'],
      dtype='object')
[('Pain